In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util
import spacy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity as cs_tfidf
from transformers import pipeline
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from datasets import load_dataset

In [ ]:
class Examiner:
    def __init__(self):
        self.model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
        self.tokenizer = T5Tokenizer.from_pretrained("t5-small")
        self.model.load_weights("/content/drive/MyDrive/ML_Matrix_hackhathon/Models/tf_model.h5")

    def preprocessing(self, text):
        vocabulary=text
        j=0
        sentences=[]
        vocabulary=vocabulary.replace('\n',' ')
        for i in range(len(vocabulary)):
            if (vocabulary[i]=='.' or vocabulary[i]=='!' or vocabulary[i]=='?'):
                sentences.append(vocabulary[j+1:i])
                j=i

        input=5  #input given by user for number of questions
        if(input<=len(sentences)):
            len(sentences)%input
        else:
            print('Enter No. of questions less than or equal to ',len(sentences))



        dit={}
        dit['ns']=len(sentences)
        for i in range(len(sentences)):
            dit[i]=sentences[i]

        return dit

    def summary(self):
        import numpy as np
        def summarize_text(text, top_k=3):
        # Load BERT model and tokenizer
        tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        model = TFAutoModel.from_pretrained('bert-base-uncased')

        # Tokenization
        sentences = sent_tokenize(text)

        # Get embeddings for multiple sentences
        embeddings = []
        for sentence in sentences:
            inputs = tokenizer(sentence, return_tensors='tf', truncation=True, padding=True)
            outputs = model(inputs)
            sentence_embedding = outputs.last_hidden_state[:, 0, :].numpy()
            embeddings.append(sentence_embedding)

        sentence_embeddings = np.vstack(embeddings)

        # Ranking sentences based on similarity to the document embedding
        doc_embedding = np.mean(sentence_embeddings, axis=0)
        doc_embedding = normalize([doc_embedding])
        sentence_embeddings = normalize(sentence_embeddings)
        scores = cosine_similarity(doc_embedding, sentence_embeddings)

        ranked_sentences = [(sentences[i], scores[0][i]) for i in range(len(sentences))]
        ranked_sentences = sorted(ranked_sentences, key=lambda x: x[1], reverse=True)

        # Picking the top_k sentences which are most relevant
        top_sentences = [sent[0] for sent in ranked_sentences[:top_k]]
        summary = ' '.join(top_sentences)

        return summary

    def generate_questions(self, input_set, max_length=64):
        questions = []
        diff = input_set['DL']
        no_questions = input_set['NQ']
        for i in range(no_questions):
            current_context = f"S{i}"
            current_context = input_set[current_context]

            if diff == 1:
                answer = ""
            elif diff == 2:
                answer = ""
            elif diff == 3:
                answer = ""

            input_text = f"generate question: {current_context} answer: {answer}"
            input_ids = self.tokenizer.encode(input_text, return_tensors="tf")

            outputs = self.model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
            generated_question = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            questions.append(generated_question)

        return questions

    def evaluate_answers(self):
    def evaluate_answers(questions_dict, answers_dict):
        # Load necessary models
        nlp = spacy.load('en_core_web_sm')
        emotion_classifier = pipeline('sentiment-analysis', model='j-hartmann/emotion-english-distilroberta-base')
        sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

        lst = []
        results = {}

        def semantic_similarity(user_answer, correct_answer):
            embeddings = sentence_model.encode([user_answer, correct_answer])
            cosine_similarity = util.cos_sim(embeddings[0], embeddings[1])
            return cosine_similarity.item()

        def named_entity_adjustment(user_answer, correct_answer, base_similarity):
            doc_user = nlp(user_answer)
            doc_correct = nlp(correct_answer)
            user_entities = [(ent.text, ent.label_) for ent in doc_user.ents]
            correct_entities = [(ent.text, ent.label_) for ent in doc_correct.ents]

            if user_entities and correct_entities:
                if user_entities == correct_entities:
                    base_similarity += 0.1
                elif len(user_entities) == len(correct_entities):
                    base_similarity += 0.05

            return min(base_similarity, 1.0)

        def tfidf_similarity(user_answer, correct_answer):
            if not user_answer.strip() or not correct_answer.strip():
                return 0

            vectorizer = TfidfVectorizer()
            try:
                vectors = vectorizer.fit_transform([user_answer, correct_answer])
                tfidf_sim = cs_tfidf(vectors[0], vectors[1])[0][0]
            except ValueError:
                tfidf_sim = 0
            return tfidf_sim

        def emotion_adjustment(user_answer, correct_answer, base_similarity):
            user_emotion = emotion_classifier(user_answer)[0]['label']
            correct_emotion = emotion_classifier(correct_answer)[0]['label']

            if user_emotion == correct_emotion:
                base_similarity += 0.1
            else:
                base_similarity -= 0.1

            return min(base_similarity, 1.0)

        def real_life_similarity(user_answer, correct_answer, base_similarity):
            user_words = user_answer.split()
            correct_words = correct_answer.split()

            similar_word_count = 0
            similarity_sum = 0

            for user_word in user_words:
                for correct_word in correct_words:
                    user_embedding = sentence_model.encode(user_word)
                    correct_embedding = sentence_model.encode(correct_word)
                    similarity = util.cos_sim(user_embedding, correct_embedding).item()

                    if similarity > 0.5:
                        similarity_sum += similarity
                        similar_word_count += 1

            if similar_word_count > 0:
                avg_similarity = similarity_sum / similar_word_count
                base_similarity += avg_similarity * 0.1
            else:
                avg_similarity = 0

            return min(base_similarity, 1.0), avg_similarity

        def evaluate_and_store_score(user_answer, correct_answer, question, question_number):
            similarity_score = semantic_similarity(user_answer, correct_answer)
            tfidf_score = tfidf_similarity(user_answer, correct_answer)
            similarity_score = named_entity_adjustment(user_answer, correct_answer, similarity_score)
            similarity_score = emotion_adjustment(user_answer, correct_answer, similarity_score)

            similarity_score, real_life_score = real_life_similarity(user_answer, correct_answer, similarity_score)

            combined_similarity = (similarity_score + tfidf_score) / 2
            combined_similarity = min(max(combined_similarity, 0), 1.0)

            lst.append(combined_similarity)

            if combined_similarity >= 0.80:
                remark = "The answer is correct."
            elif combined_similarity >= 0.65:
                remark = "The answer is similar in some cases."
            elif combined_similarity >= 0.40:
                remark = "The answer is similar in general terms."
            else:
                remark = "The answer is wrong or not similar."

            percentage = combined_similarity * 100

            results[f'question{question_number}'] = {
                'question': question,
                'combined_score': combined_similarity,
                'percentage': percentage,
                'remark': remark,
                'real_life_similarity': real_life_score
            }

        def plot_scores():
            plt.figure(figsize=(10, 6))
            plt.plot(lst, marker='o', color='b')
            plt.title('Combined Similarity Scores Over Time')
            plt.xlabel('Attempt')
            plt.ylabel('Similarity Score')
            plt.grid(True)
            file_path = 'similarity_scores_plot.jpg'
            plt.savefig(file_path, format='jpg', bbox_inches='tight')
            return file_path

        # Evaluate answers
        for i in range(1, questions_dict['nq'] + 1):
            correct_answer = questions_dict.get(f'a{i}', '').strip()
            user_answer = answers_dict.get(f'uan{i}', '').strip()

            if correct_answer:
                evaluate_and_store_score(user_answer, correct_answer, questions_dict.get(f'q{i}', ''), i)
            else:
                question_text = questions_dict.get(f'q{i}', '').strip()
                evaluate_and_store_score(user_answer, question_text, question_text, i)

        # Plot scores
        image_path = plot_scores()

        return results, image_path, lst



In [ ]:




# Example usage:
questions = {
    'nq': 3,
    'q1': 'What is the capital of France?',
    'a1': 'Paris',
    'q2': 'Who wrote Hamlet?',
    'a2': 'Shakespeare',
    'q3': 'What is 2 + 2?',
    'a3': '4',
}

answers = {
    'uan1': 'Paris',
    'uan2': 'Shakespeare',
    'uan3': '4'
}

results, image_path, scores = evaluate_answers(questions, answers)

print(f"\nThe graph has been saved at: {image_path}")
print("\nResults for each question:")
for key, value in results.items():
    print(f"{key}: {value}")
print(f"\nScores: {scores}")